In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)


In [2]:
data = pd.read_csv("Qmerge1-FirstLastHRV.csv") 
data.head()

,nsrrid,gender_x,race_x,age_s1_x,ang_date,chd_dthdt,chf_date,cvd_dthdt,mi_date,stk_date,...,ihr,NN_RR,AVNN,SDNN,VLF,LF,LF_n,HF,HF_n,LF_HF
0,200079,2,3,56,NaN,NaN,NaN,NaN,NaN,NaN,...,75.112951,0.992000,798.797,46.2118,695.6390,285.530,0.256639,827.0460,0.743361,0.345241
1,200079,2,3,56,NaN,NaN,NaN,NaN,NaN,NaN,...,70.633025,0.968839,849.461,72.6343,1234.8283,1666.850,0.507907,1614.9500,0.492093,1.032140
2,200082,1,1,40,NaN,NaN,NaN,NaN,NaN,NaN,...,59.355400,0.996610,1010.860,39.4332,739.2121,531.024,0.686206,242.8310,0.313794,2.186800
3,200082,1,1,40,NaN,NaN,NaN,NaN,NaN,NaN,...,56.666320,0.996466,1058.830,146.8230,13231.3000,3964.450,0.378345,6513.9400,0.621655,0.608610
4,200108,2,1,60,NaN,NaN,NaN,NaN,NaN,NaN,...,64.954397,0.990741,923.725,27.7213,532.0790,168.406,0.672796,81.9015,0.327204,2.056200


# Column Names description¶

nsrrid = This subject identifier was created by the National Sleep Research Resource (NSRR) team to more easily match with file downloads. Use 'pptid' to link with Biologic Specimen and Data Repository Information Coordinating Center (BioLINCC) and cohort-specific data.

BLUE25 = Quality of Life (QOL) (Sleep Heart Health Study Visit One (SHHS1)): Felt downhearted and blue.

 1: All of the time
 2: Most of the time
 3: A good bit of the time
 4: Some of the time
 5: A little of time
 6: None of the time
ihr = Instantaneous heart rate

NN_RR = Ratio of consecutive normal sinus beats (NN) over all cardiac inter-beat (RR) intervals

AVNN = Mean of all normal sinus to normal sinus interbeat intervals (NN)

SDNN = Standard deviation of all normal sinus to normal sinus interbeat (NN) intervals

VLF = Very low frequency power: the normal sinus to normal sinus interbeat (NN) interval spectral power between 0.003 and 0.04 Hz

LF = Low frequency power: the normal sinus to normal sinus interbeat (NN) interval spectral power between 0.04 and 0.15 Hz

HF = High frequency power: the normal sinus to normal sinus interbeat (NN) interval spectral power between 0.15 and 0.4 Hz

HF_n = High frequency power (normalized)

LF_HF = The ratio of low to high frequency power

In [3]:
heart_rate_data = data.copy()


In [4]:
heart_rate_data = heart_rate_data[['nsrrid', 'gender_x', 'race_x', 'age_s1_x', 'BLUE25', 'ihr', 'NN_RR', 'AVNN', 'SDNN', 'VLF', 'LF', 'HF', 'HF_n', 'LF_HF']]

In [5]:
heart_rate_data.head()

,nsrrid,gender_x,race_x,age_s1_x,BLUE25,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
0,200079,2,3,56,5.0,75.112951,0.992000,798.797,46.2118,695.6390,285.530,827.0460,0.743361,0.345241
1,200079,2,3,56,5.0,70.633025,0.968839,849.461,72.6343,1234.8283,1666.850,1614.9500,0.492093,1.032140
2,200082,1,1,40,6.0,59.355400,0.996610,1010.860,39.4332,739.2121,531.024,242.8310,0.313794,2.186800
3,200082,1,1,40,6.0,56.666320,0.996466,1058.830,146.8230,13231.3000,3964.450,6513.9400,0.621655,0.608610
4,200108,2,1,60,5.0,64.954397,0.990741,923.725,27.7213,532.0790,168.406,81.9015,0.327204,2.056200


In [6]:
# Skipping every other row, to reemove duplicate rows
heart_rate_data = heart_rate_data[::2]
heart_rate_data.head()

,nsrrid,gender_x,race_x,age_s1_x,BLUE25,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
0,200079,2,3,56,5.0,75.112951,0.992000,798.797,46.21180,695.6390,285.53000,827.04600,0.743361,0.345241
2,200082,1,1,40,6.0,59.355400,0.996610,1010.860,39.43320,739.2121,531.02400,242.83100,0.313794,2.186800
4,200108,2,1,60,5.0,64.954397,0.990741,923.725,27.72130,532.0790,168.40600,81.90150,0.327204,2.056200
6,200109,2,1,71,6.0,76.865428,0.997389,780.585,8.41453,55.3561,8.87934,4.38036,0.330351,2.027080
8,200112,1,1,48,5.0,62.348286,0.996785,962.336,88.53440,3987.3524,3269.69000,292.34900,0.082073,11.184200


In [7]:
# Renaming columns 
heart_rate_data.rename(index=str, columns={'gender_x': 'Gender', 'race_x': 'Race', 'age_s1_x': 'Age'})
heart_rate_data.head()

,nsrrid,gender_x,race_x,age_s1_x,BLUE25,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
0,200079,2,3,56,5.0,75.112951,0.992000,798.797,46.21180,695.6390,285.53000,827.04600,0.743361,0.345241
2,200082,1,1,40,6.0,59.355400,0.996610,1010.860,39.43320,739.2121,531.02400,242.83100,0.313794,2.186800
4,200108,2,1,60,5.0,64.954397,0.990741,923.725,27.72130,532.0790,168.40600,81.90150,0.327204,2.056200
6,200109,2,1,71,6.0,76.865428,0.997389,780.585,8.41453,55.3561,8.87934,4.38036,0.330351,2.027080
8,200112,1,1,48,5.0,62.348286,0.996785,962.336,88.53440,3987.3524,3269.69000,292.34900,0.082073,11.184200


In [8]:
# Renaming columns and getting rid of Nan
heart_rate_data = heart_rate_data.rename(index=str, columns={'gender_x': 'Gender', 'race_x': 'Race', 'age_s1_x': 'Age'})
heart_rate_data = heart_rate_data[heart_rate_data['BLUE25'] != 'NAN']
heart_rate_data.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,nsrrid,Gender,Race,Age,BLUE25,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
0,200079,2,3,56,5.0,75.112951,0.992000,798.797,46.21180,695.6390,285.53000,827.04600,0.743361,0.345241
2,200082,1,1,40,6.0,59.355400,0.996610,1010.860,39.43320,739.2121,531.02400,242.83100,0.313794,2.186800
4,200108,2,1,60,5.0,64.954397,0.990741,923.725,27.72130,532.0790,168.40600,81.90150,0.327204,2.056200
6,200109,2,1,71,6.0,76.865428,0.997389,780.585,8.41453,55.3561,8.87934,4.38036,0.330351,2.027080
8,200112,1,1,48,5.0,62.348286,0.996785,962.336,88.53440,3987.3524,3269.69000,292.34900,0.082073,11.184200


In [9]:
dep_data = heart_rate_data.dropna(axis=0, how='any')
dep_data.shape[0]

dep_data.loc[dep_data['BLUE25'] < 5, 'BLUE25'] = 0
dep_data.loc[dep_data['BLUE25'] >= 5, 'BLUE25'] = 1
dep_data.head(2)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,nsrrid,Gender,Race,Age,BLUE25,ihr,NN_RR,AVNN,SDNN,VLF,LF,HF,HF_n,LF_HF
0,200079,2,3,56,1.0,75.112951,0.99200,798.797,46.2118,695.6390,285.530,827.046,0.743361,0.345241
2,200082,1,1,40,1.0,59.355400,0.99661,1010.860,39.4332,739.2121,531.024,242.831,0.313794,2.186800


In [10]:
dep_data.dtypes
#dep_data

nsrrid      int64
Gender      int64
Race        int64
Age         int64
BLUE25    float64
ihr       float64
NN_RR     float64
AVNN      float64
SDNN      float64
VLF       float64
LF        float64
HF        float64
HF_n      float64
LF_HF     float64
dtype: object

In [11]:
#to show how many records and fields we have now in the cleaned dataset
print(dep_data.shape)

(449, 14)


In [13]:
X = dep_data.drop('BLUE25', axis = 1)  #X = df1.drop('ql209f', axis = 1)
y = dep_data['BLUE25']



y_new = dep_data['BLUE25'].unique()
#print(f"Labels: {y[:10]}")
#print(f"Data: {X[:10]}")

In [14]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
classifier.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9017857142857143
Testing Data Score: 0.8407079646017699


In [18]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
652,1.0,1.0
624,1.0,1.0
378,1.0,0.0
62,1.0,1.0
466,1.0,1.0
478,1.0,1.0
752,1.0,1.0
346,1.0,1.0
596,1.0,1.0
8,1.0,1.0


In [19]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
label_encoder.fit(y_new)
encoded_y = label_encoder.transform(y_new)
encoded_y

array([1, 0])

In [20]:
for label, original_class in zip(encoded_y, y_new):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: 1.0
Encoded Label: 1
------------
Original Class: 0.0
Encoded Label: 0
------------


In [21]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

Using TensorFlow backend.


array([[0., 1.],
       [1., 0.]], dtype=float32)

In [22]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [23]:
classifier.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [24]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.339369
         Iterations 7
                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.041   
Dependent Variable: BLUE25           AIC:              330.7533
Date:               2019-05-09 19:54 BIC:              384.1446
No. Observations:   449              Log-Likelihood:   -152.38 
Df Model:           12               LL-Null:          -158.92 
Df Residuals:       436              LLR p-value:      0.36245 
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     7.0000                                     
----------------------------------------------------------------
             Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
----------------------------------------------------------------
nsrrid       0.0001    0.0000   1.4725  0.1409  -0.0000   0.0002
Gender       0.1569    0.3258   0.4817  0.6300  -0.4817   0.7955
Race        -0.5154   

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [26]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.87


In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[  0  18]
 [  0 117]]


In [ ]:
#heart_rate_data.to_csv(r'HRV\heart_rate_data.csv', index=False)
